In [15]:
#Buliding nodes from runeterra metajson.

# Builds nodes with champion asset links, regions, and archetypes names from MR meta report


In [16]:
import json

with open("data/eternal-meta-data.json", "r") as f:
    input_data = json.load(f)
input_data = input_data["stats"]["three"]["americas"]
input_data

[{'archetype': 'Azir Irelia (IO/SH)',
  'assets': {'regions': ['IO', 'SH'],
   'champions': [['Azir', '04SH003', 'shurima'],
    ['Irelia', '04IO005', 'ionia']]},
  'total_matches': 1786,
  'playrate': 4.99,
  'winrate': 57.75,
  'best_decks': 'CMCACBQHEIBACAQMFIBAIBYDGMCAIAQFBAEQ4AYBAMBAKAQEAIAQIAYEA4NDMXIBAECAO6I,37,69.44,2.07/CMBQEAICAYVAIBACAECQQCIEAQDQGGRTOICQCAICBQAQEAQKAEBQEFABAQBAIAIEA5OQA,158,64.90,8.85/CMCAEAICAYVAEBQHAURAGBACAQCQSBAEA4BRUM3SAMAQCAQMAEBAECQBAQDV2AIBAEBDC,378,59.43,21.16/CMBQEAICAYVAIBACAQCQQCIFAQDQGGRTGZOQEAIBAIGACAYCCQBQCAQCBIAQIAQLAECAO6I,71,58.21,3.98/CMBQGAICAYGCUBAEAIAQIBIJAQCAOAY2GNZAEAIEAIEACBAHLUBQCAICGEAQEAQKAEBQEFA,224,54.81,12.54',
  'slug': 'azir-irelia-io-sh',
  'guide': 'https://masteringruneterra.com/lor-deck-guide-azir-irelia/'},
 {'archetype': 'Nilah Twisted Fate (BW/PZ)',
  'assets': {'regions': ['BW', 'PZ'],
   'champions': [['Nilah', '08BW008', 'bilgewater'],
    ['Twisted Fate', '02BW026', 'bilgewater']]},
  'total_matches': 1308,
  'play

In [17]:
from function_pipes import pipe
top_15 = input_data[:15]
output = []

def parse_rate(rate):
    return round(rate / 100, 4)

def parse_data(i, data):
    ans = {"label": data["archetype"], 
           'assets': data['assets'],
           "id": f"n{i}",
           "play_rate": parse_rate(data["playrate"]),
            "win_rate": parse_rate(data["winrate"]),
           "games_played": data["total_matches"]
           }
    return ans

parsed_data = [parse_data(i, data) for i, data in enumerate(top_15)]

In [21]:
with open("data/nodes.json", "w") as f:
    json.dump(parsed_data, f)

# Downloading Champion Portraits

Getting champions we already have

In [19]:
from os import listdir
from os.path import isfile, join
def get_already_seen():
    path = "data/images/champion_portraits"
    only_files = [f for f in listdir(path) if isfile(join(path, f))]
    champion_codes = [f.split(".")[0] for f in only_files]
    return champion_codes
print(get_already_seen())

['01IO041', '05BC058', '04SH019', '06NX012', '05DE009', '05PZ006', '03DE011', '01PZ008', '04NX004', '05BC029', '06RU002', '05BW005', '02BW053', '08FR001', '07IO011', '02SI008', '08PZ001', '06PZ021', '06SI012', '08MT003', '01FR038', '08BW008', '01IO009', '08SI042', '04BW005', '08RU014', '01PZ040', '03MT087', '04SI055', '06IO008']


In [20]:
import requests
#TODO: init to logs we have
already_seen = set(get_already_seen())

def download_portrait(champion):
    champion_code = champion[1]
    if champion_code in already_seen:
        return
    url = f"https://masteringruneterra.com/wp-content/plugins/deck-viewer/assets/images/champions/{champion_code}.webp"
    res = requests.get(url)
    with open(f"data/images/champion_portraits/{champion_code}.webp", "wb") as f:
        f.write(res.content)
    already_seen.add(champion_code)

for data in parsed_data:
    for champion in data["assets"]["champions"]:
        download_portrait(champion)